In [35]:
from enum import Enum
from functools import total_ordering
from dataclasses import dataclass

with open('input.txt') as f:
  unparsed_hands = f.read().splitlines()

@total_ordering
class HandType(Enum):
  HIGH_CARD = 1
  ONE_PAIR = 2
  TWO_PAIR = 3
  THREE_OF_A_KIND = 4
  FULL_HOUSE = 5
  FOUR_OF_A_KIND = 6
  FIVE_OF_A_KIND = 7

  def __lt__(self, other):
    if self.__class__ is other.__class__:
      return self.value < other.value
    return NotImplemented

@dataclass
@total_ordering
class Hand:
  cards: str
  bid: int

  card_values = {
    'J': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    'T': 10,
    'Q': 11,
    'K': 12,
    'A': 13
  }

  def __lt__(self, other):
    if self.__class__ is other.__class__:
      if self.type == other.type:
        return list(map(lambda c: self.card_values[c], self.cards)) < list(map(lambda c: other.card_values[c], other.cards))
      return self.type < other.type


  @property
  def letter_count(self) -> dict[str,int]:
    letter_count = {}
    for c in self.cards:
      letter_count[c] = letter_count.get(c, 0) + 1
    return letter_count

  @property
  def type(self) -> HandType:
    return self.type_with_jokers()

  def type_with_jokers(self) -> HandType:
    joker_count = self.letter_count.get('J', 0)
    if joker_count == 5:
      return HandType.FIVE_OF_A_KIND
    count_wihout_jokers = {key: value for key, value in self.letter_count.items() if key != 'J'}
    max_key = max(count_wihout_jokers, key=lambda k: count_wihout_jokers[k])
    count_wihout_jokers[max_key] += joker_count
    return self.type_without_jokers(count_wihout_jokers)

    # 1 2 3 


  
  @staticmethod
  def type_without_jokers(letter_count) -> HandType:
    if 5 in letter_count.values():
      return HandType.FIVE_OF_A_KIND
    if 4 in letter_count.values():
      return HandType.FOUR_OF_A_KIND
    if 3 in letter_count.values():
      if 2 in letter_count.values():
        return HandType.FULL_HOUSE
      return HandType.THREE_OF_A_KIND
    pair_count = list(letter_count.values()).count(2)
    if pair_count == 2:
      return HandType.TWO_PAIR
    if pair_count == 1:
      return HandType.ONE_PAIR
    return HandType.HIGH_CARD
    

hands = []
for unparsed_hand in unparsed_hands:
  cards, bid_str = unparsed_hand.split()
  hands.append(Hand(cards, int(bid_str)))

# hands = [
#   Hand('32T3K', 765),
#   Hand('T55J5', 684),
#   Hand('KK677', 28),
#   Hand('KTJJT', 220),
#   Hand('QQQJA', 483)
# ]

sorted_hands = sorted(hands)

res = 0
for i, hand in enumerate(sorted_hands):
  res += hand.bid * (i + 1)

print(res)

249781879
